In [2]:
import pandas as pd
import utils
import plotly.graph_objects as go 
import instrument 


In [3]:
pair = "GBP_JPY"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [4]:
df = pd.read_pickle(utils.get_his_data_filename(pair,granularity))
non_cols = ['time','volume']
mod_cols = [ x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)


In [5]:
df_ma = df[['time','mid_o','mid_h','mid_l','mid_c']].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [6]:
df_ma.head(100)

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2021-09-30T23:00:00.000000000Z,150.110,150.148,150.070,150.130,150.273313,150.606297
1,2021-10-01T00:00:00.000000000Z,150.128,150.177,149.876,149.876,150.237688,150.570641
2,2021-10-01T01:00:00.000000000Z,149.880,149.921,149.600,149.822,150.189437,150.538484
3,2021-10-01T02:00:00.000000000Z,149.824,149.873,149.662,149.690,150.138187,150.509016
4,2021-10-01T03:00:00.000000000Z,149.692,149.774,149.546,149.560,150.071812,150.478203
...,...,...,...,...,...,...,...
95,2021-10-06T22:00:00.000000000Z,151.368,151.448,151.362,151.421,151.215500,151.373703
96,2021-10-06T23:00:00.000000000Z,151.424,151.457,151.342,151.429,151.213063,151.384000
97,2021-10-07T00:00:00.000000000Z,151.427,151.507,151.260,151.410,151.226062,151.393672
98,2021-10-07T01:00:00.000000000Z,151.408,151.489,151.340,151.446,151.245937,151.398672


In [7]:
df_ma.head(1000)

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2021-09-30T23:00:00.000000000Z,150.110,150.148,150.070,150.130,150.273313,150.606297
1,2021-10-01T00:00:00.000000000Z,150.128,150.177,149.876,149.876,150.237688,150.570641
2,2021-10-01T01:00:00.000000000Z,149.880,149.921,149.600,149.822,150.189437,150.538484
3,2021-10-01T02:00:00.000000000Z,149.824,149.873,149.662,149.690,150.138187,150.509016
4,2021-10-01T03:00:00.000000000Z,149.692,149.774,149.546,149.560,150.071812,150.478203
...,...,...,...,...,...,...,...
995,2021-11-29T13:00:00.000000000Z,151.500,151.631,151.410,151.550,151.400563,152.512063
996,2021-11-29T14:00:00.000000000Z,151.548,151.698,151.330,151.541,151.411250,152.476141
997,2021-11-29T15:00:00.000000000Z,151.538,151.574,151.010,151.054,151.370000,152.432609
998,2021-11-29T16:00:00.000000000Z,151.060,151.333,151.036,151.186,151.345875,152.390297


In [8]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0
        

In [17]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)


In [18]:
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()
df_trades['DELTA'] = ( df_trades.mid_c.diff() / i_pair.pipLocation ).shift(-1)
df_trades['GAIN'] = df_trades['DELTA'] * df_trades['IS_TRADE']


In [22]:
from dateutil.parser import *
df_trades['time'] = [parse(x) for x in df_trades.time] 

In [30]:
df_trades['DURATION'] = df_trades.time.diff().shift(-1)
df_trades['DURATION'] = [x.total_seconds() / 3600 for x in df_trades.DURATION]

In [31]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 22 to 3928
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   time       68 non-null     datetime64[ns, tzutc()]
 1   mid_o      68 non-null     float64                
 2   mid_h      68 non-null     float64                
 3   mid_l      68 non-null     float64                
 4   mid_c      68 non-null     float64                
 5   MA_16      68 non-null     float64                
 6   MA_64      68 non-null     float64                
 7   DIFF       68 non-null     float64                
 8   DIFF_PREV  68 non-null     float64                
 9   IS_TRADE   68 non-null     int64                  
 10  DELTA      67 non-null     float64                
 11  GAIN       67 non-null     float64                
 12  DURATION   67 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(11), int64

In [33]:
df_trades[['time','DURATION']].tail(25)

,time,DURATION
2416,2022-02-18 18:00:00+00:00,106.0
2474,2022-02-23 04:00:00+00:00,17.0
2491,2022-02-23 21:00:00+00:00,118.0
2561,2022-02-28 19:00:00+00:00,7.0
2568,2022-03-01 02:00:00+00:00,44.0
2612,2022-03-02 22:00:00+00:00,33.0
2645,2022-03-04 07:00:00+00:00,116.0
2713,2022-03-09 03:00:00+00:00,48.0
2761,2022-03-11 03:00:00+00:00,7.0
2768,2022-03-11 10:00:00+00:00,439.0


In [13]:
df_trades['GAIN'].sum()

2003.800000000001

In [14]:
df_plot = df_ma.iloc[0:500].copy()

In [15]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1),opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C",
))
for ma in ma_list: 
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(
    width=1700,
    height=650,
    margin=dict(t=10,r=10,b=10,l=10),
    font=dict(size=10, color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,
    fixedrange=True,
    rangeslider=dict(visible=False) 
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True,
    fixedrange=True,
)
fig.show()